# Projekt "Self driving cars"
## Algorytmy Ewolucyjne 2017/2018
### Prowadzący: Piotr Lipiński
### Autor: Mateusz Hazy

## Opis projektu

Celem projektu "self driving cars" było stworzenie modelu systemu sterowania autem podejmującego decyzje na podstawie informacji, które prawdziwe auto mogłoby dostarczać w rzeczywistości. 
W uproszczonym modelu przyjęto, że auto posiada 5 czujników umieszczonych na przedniej masce pod różnymi kątami, które podają odległość do najbliższej przeszkody. Samo auto jest prostokątem o zadanych wymiarach, a jego ruch jest ograniczony przez maksymalną prędkość, przyspieszenie/hamowanie, zakres skrętu. Nie są jednak brane pod uwagę tarcie i inne własności fizyczne. Niemniej jednak model dobrze obrazuje reagowanie auta na napotykane przeszkody i dostosowywanie się do kształtu trasy.

## Środowisko i symulacja

Ruch auta symulowany jest na płaszczyźnie na podstawie aktualnej prędkości i położenia jako liniowy w małym odstępie czasu.

$$ X, Y \text{- położenie auta} $$
$$ V \text{-  prędkość, } \Delta V \text{- przyspieszenie, }$$
$$ \alpha \text{- odchylenie auta od poziomej prostej} , \Delta \alpha \text{- zmiana kąta}  $$

$$ \alpha(t +  \Delta t ) = \alpha(t) + \Delta \alpha(t + \Delta t) $$
$$ V(t + \Delta t) =  V(t) +  \Delta V(t+  \Delta t) $$
$$ X(t + \Delta  t) = X(t)  + \Delta t \cdot V(t +  \Delta t) \cdot \cos{ \alpha(t + \Delta t)} $$
$$ Y(t + \Delta  t) = Y(t)  + \Delta t \cdot V(t +  \Delta t) \cdot \sin{ \alpha(t + \Delta t)} $$

Jak widać, system sterowania autem musi podjąć decyzję o wartościach $ \Delta V $ oraz $ \Delta \alpha $ w każdej chwili $t$

Trasa, po której porusza się auto jest ograniczona łamanymi. Cały ruch odbywa się w dwóch wymiarach.

## System sterowania

Za podejmowanie decyzji dotyczących sterowania auta będzie odpowiedzialna pewna sieć neuronowa, biorąca na wejście odległości od przeszkód poszczególnych czujników oraz akualną prędkość auta. Na wyjście zostaną zwrócone, odpowiednio przeskalowane wartości $ \Delta V $ oraz $ \Delta \alpha$.

Należy tu jednak zaznaczyć, że przez sieć neuronową można rozumieć nie tylko tradycyjną sieć o regularnej budowie (każdy wierzchołek i-tej warstwy jest połączony z każdym z i+1-szej), ale też dowolny skierowany graf acykliczny o tej wlasności, że pierwsze k wierzchołków (wierzchołki wejściowe, w tym przypadku 6) ma stopień wejściowy równy 0, a ostatnie l ( wierzchołki wyjściowe, tutaj 2) mają stopień wyjściowy równy 0. 

## Regularne sieci neuronowe 

Zajmiemy się rozwojem systemu sterującego opartego o regularną sieć neuronową:
- Ustalona liczba warstw
- W każdej warstwie ustalona liczba wierzchołków
- Każdy wierzchołek z i-tej warstwy jest połączony z każdym wierzchołkiem z i+1-szej
- W każdym wierzchołku v obliczamy wartość biorąc kombinację liniową (X) wartości z wierzchołków wchodzących (u) do v ze wsþółczynnikami będacymi wagami odpowiednich krawędzi (u, v). Wartością w wierzchołku v będzie $ \tanh{X}$. 

Będziemy znajdować optymalne wagi na krawędziach dla ustalonej struktury sieci neuronowej. Wykorzystamy do tego strategie ewolucyjne $ ES ( \mu + \lambda) $ 

Dobieranie struktury sieci oraz parametry algorytmu $ES(\mu + \lambda)$ będzie empiryczne. Pozostaje odpowiednio zdefiniować funkcję celu.
- Należy znaleźć zestaw danych do uczenia - niech będzie to $n$ losowych torów - $T_i$
- Dla każdego toru wprowadzimy limit czasu na jego pokonanie - $t_i$
- $ D(T_i)$ długość toru $T_i$
- $D(T_i, N)$ długość pokonana prez auto sterowane siecią $N$ na torze $T_i$
- $T(T_i, N)$ czas w jakim auto sterowane siecią $N$ pokonało tor $T_i$

Określmy funkcję celu $Fitness_i$ jako fragment toru $T_i$ pokonany przez auto sterowane daną siecią:
$$ Fitness_i(N) = \frac{ D(T_i, N)}{D(T_i)}  $$

Warto uwzględnić, czy auto się rozbiło, zabrakło mu czasu na pokonanie trasy, bądź pokonało trasę przed czasem. Dlatego funkcję celu jako:
$$ Fitness(N) = \frac{1}{n} \sum_{i=1}^{n} Fitness_i(N) \cdot C(N, T_i) $$ 
gdzie
$$ C(N, T_i) = 
\begin{cases}
                0.8 \text{, gdy auto sterowane przez N rozbiło sie na torze T_i.} \\
                1 + \frac{t_i - T(T_i, N)}{2 \cdot t_i} \text{, gdy auto sterowane przez N pokonało tor T_i przed czasem.} \\
                1 \text{, w p. p.}\\
\end{cases}$$

## Testy

Można się spodziewać, że wyewoluowane sieci będą dobrze radziły sobie na torach z zestawu testowego, jednak nie wiadomo, czy taki system nauczył się jeździć po torach czy jeździć po torach z zestawu testowego. Dlatego najlepszy system po zakończeniu algorytmu będziemy testować na pewnych torach "benchmarkowych".

Animacje prezentujące proces ewolucji pokazują 5 najlepszych systemów z pokoleń, w których nastąpiła wyraźna poprawa wartości funkcji $Fitness$

Grafiki prezentujące wygląd sieci należy interpretować następująco:
- kolor krawędzi oznacza znak (niebieski - ujemny, czerwony - dodatni) 
- grubość krawędzi oznacza wielkość wagi
- Najniższy wierzchołek pierwszej warstwy to prędkość auta, pozostałe - wartości z czujników
- Górny wierzchołek ostatniej warstwy to $\Delta V$, dolny - $\Delta \alpha$

### Struktura sieci (6, 8, 4, 2), populacja 50, 100 iteracji

#### Nauczanie

<center><video controls width="900" height="500" src="images/sim1/LEARNING.mp4" /></center>

<center><img width="600" height="400" src="images/sim1/results.png" /></center>

#### Najlepsza sieć

<center><img width="600" height="400" src="images/sim1/brain.png" /></center>

#### Benchmarki

<center><video controls width="900" height="500" src="images/sim1/BENCHMARKS.mp4" /></center>

### Struktura sieci (6, 8, 2), populacja 50, 100 iteracji

#### Nauczanie

<center><video controls width="900" height="500" src="images/sim2/LEARNING.mp4" /></center>

<center><img width="600" height="400" src="images/sim2/results.png" /></center>

#### Najlepsza sieć

<center><img width="600" height="400" src="images/sim2/brain.png" /></center>

#### Benchmarki

<center><video controls width="900" height="500" src="images/sim2/BENCHMARKS.mp4" /></center>

## Skierowane grafy acykliczne

Wadą poprzedniego rozwiązania było to, że z góry ustalana była struktura sieci neuronowej. Co prawda te regularne struktury są najczęściej stosowaną i najlepiej sprawdzoną prostą metodą stosowana w tego typu problemach. Jednak skąd właściwie wiadomo, jaka powinna być optymalna liczba warstw i liczba wierzchołków w każdej z nich? Dlatego warto spróbować stworzyć algorytm znajdujący optymalną sieć w znacznie poszerzonej przestrzeni poszukiwań: zmienne będą nie tylko wagi na krawędziach, ale też cała struktura sieci. Jedynym ograniczeniem będzie maksymalna liczba wierzchołków w sieci i to, że musi tworzyć skierowany graf acykliczny. Będziemy je nazywać sieciami DAG.

Obliczanie wartości w wierzchołkach sieci będzie oparte na tej samej zasadzie co w regularnej sieci neuronowej. Warto jednak zwrócić uwagę, że aby poprawnie wyliczyć wszystkie wartości należy rozpatrywać wierzchołki w kolejności topologicznej.

### Krzyżowanie grafów

Mając ograniczenie na liczbę wierzchołków w sieci (V), strukturę grafu można pamiętać jako macierz sąsiedztwa o wymiarach V x V. Będziemy chcieli krzyżować takie macierze analogicznie do tego jak krzyżowane są wektory w operatorze krzyżowania jednopunktowego.

Wybierzmy punkt cięcia $ c \in [0, n] $
- Pierwsze c wierszy pierwszego rodzica dajemy do pierwszego potomka, pozostałe wiersze do drugiego.
- Analogicznie postępujemy z drugim rodzicem, tym samym tworząc dwóch potomków z dwóch rodziców.

### Krzyżowanie wag

Z uwagi na znacznie większą przestrzeń poszukiwań w porównaniu do poprzedniego przypadku, postanowiłem wzbogacić algorytm o operator krzyżowania wag. Użyłem operatora SBX służacego do krzyżowania genów o wartościach rzeczywistych opisanego w poniższym artykule: 
 - http://wpmedia.wolfram.com/uploads/sites/13/2018/02/09-2-2.pdf
 - https://www.slideshare.net/paskorn/simulated-binary-crossover-presentation?type=powerpoint

### Mutacje

Główną ideą algorytmu dalej ma być $ES(\mu + \lambda)$. Należy więc mutować nie tylko wagi, ale również sama strukturę. Z powodu dość małych populacji używanych w algorytmie postanowiłem uprościć strategię ewolucyjną. Mianowicie, zamiast osobnej wartości odpowiedzialnej za wielkość mutacji dla każdego genu, osobnik będzie posiadać 2 wartości:
 - $ \sigma $ - wartość odpowiedzialna za mutacje wag 
 - $ \rho $ - wartość odpowiedzialna za mutacje krawędzi
 
Mutacja wag odbywa sie tak samo jak w zwykłym algorytmie ES. Jednak krawędzie przyjmują wartości 0 lub 1. Dlatego wartość $\rho$ określa nie wielkość mutacji, ale jej prawdopodobieństwo. Podobnie jak $\sigma$, jest ona zmieniana w mutacji. Ideą jest to, że osobniki posiadające dobrą strukturę sieci będa miały małą wartość $\rho$ aby nie zmienić na gorszą i odwrotnie.

Istotny będzie wybór początkowych wartości $\rho$. Przyjąłem, że bezpieczną będzie $\rho = \frac{1}{V}$

## Testy

Grafiki reprezentujące sieci zostały nieco zmodyfikowane:
- Zielony wierzchołek reprezentuje $ \Delta \alpha $, a różowy - $ \Delta V$ 
- Fioletowe wierzchołki reprezentują wierzchołki wejściowe

Stuktury sieci prezentowane w animacje pokazują sieci, które były najlepsze przez pewien okres ewolucji.

Z uwagi na znacznie większą przestrzeń poszukiwań, wielkość populacji w testowaniu algorytmu jest większa niż w przypadku regularnych sieci.

### Sieć złożona z 22 wierzchołków, populacja 100, 100 iteracji, próba nr 1

#### Nauczanie

<center><video controls width="900" height="500" src="images/simdag1/LEARNING.mp4" /></center>

<center><img width="600" height="400" src="images/simdag1/results.png" /></center>

#### Najlepsza sieć

<center><video controls width="600" height="400" src="images/simdag1/BRAINS.mp4" /></center>

#### Benchmarki

<center><video controls width="900" height="500" src="images/simdag1/BENCHMARKS.mp4" /></center>

### Sieć złożona z 22 wierzchołków, populacja 100, 100 iteracji, próba nr 2

#### Nauczanie

<center><video controls width="900" height="500" src="images/simdag2/LEARNING.mp4" /></center>

<center><img width="600" height="400" src="images/simdag2/results.png" /></center>

#### Najlepsza sieć

<center><video controls width="600" height="400" src="images/simdag2/BRAINS.mp4" /></center>

#### Benchmarki

<center><video controls width="900" height="500" src="images/simdag2/BENCHMARKS.mp4" /></center>

## Porównanie metod

Eksperyment można uznać za udany, jednak wystąpiło kilka ciekawych zjawisk:
- Auta bardzo płynnie poruszają się po torach z zestawu uczącego, jednak w przypadku obu sieci jazda aut na prostych benchmarkach jest chaotyczna.
- W przypadku regularnych sieci, jazda auta sterowanego przez sieć złożoną z większej liczby warstw jest bardziej stabilna
- W przypadku sieci DAG, uzyskane sieci są istotnie różne w różnych wywołaniach algorytmu z tymi samymi parametrami.
- W sieciach DAG algorytm z początku często zmienia strukturę grafu, jednak pod koniec następują jedynie drobne modyfikacje (dodanie lub usunięcie kilku krawędzi). Dobrze widać to w przypadku próby nr 1.
- Trudno stwierdzić, która z metod jest skuteczniejsza. Biorąc pod uwagę czas obliczeń, pierwsza metoda jest skuteczniejsza. Jednak przy dużej mocy obliczeniowej, można uczyć auta na znacznie większym zestawie danych i większej populacji. Wtedy sieci DAG mogą okazać się lepsze, ze względu na ich większą ogólność.
- W przypadku obliczeń z populacją wielkości 50-100 obie metody są porównywalnie skuteczne.

## Dalszy rozwój

- Wadą obu rozwiązań jest to, że systemy nie mogą się uczyć na nowych torach. Dobrym pomysłem byłoby zmodyfikowanie sieci tak, by mogła się uczyć na nowych torach nie tracąc umiejętności jazdy na poprzednich.
- Symulacja jazdy jest mocno uproszczona, warto sprawdzić, czy w bardziej realistycznym modelu metody będą dalej działać.